# Cargar CSV a Base de Datos Spotify

In [6]:
pip install pandas mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import mysql.connector

# Cargar los archivos
df_artistas_info = pd.read_csv('spotify_base_datos_info_artistas_todos.csv')
df_canciones = pd.read_csv('spotify_base_datos_canciones_todas.csv')

# Eliminar duplicados de artistas para evitar errores de inserción
df_artistas_info = df_artistas_info.drop_duplicates(subset=['artista'])

In [8]:
# Conectar con tu servidor MySQL.

def conectar():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="AlumnaAdalab",
        database="bd_spotify"
    )

conexion = conectar()
cursor = conexion.cursor()

In [9]:
# Extraemos los géneros únicos de la tabla de canciones e insertamos en la tabla generos
generos_unicos = df_canciones['genre'].unique()

for genero in generos_unicos:
    try:
        cursor.execute("INSERT IGNORE INTO generos (nombre_genero) VALUES (%s)", (genero,))
    except Exception as e:
        print(f"Error insertando género {genero}: {e}")

conexion.commit()
print("Géneros insertados.")

Géneros insertados.


In [10]:
#Informacion detallada de los artistas

for _, row in df_artistas_info.iterrows():
    sql = """INSERT INTO artistas (nombre, biografia, oyentes, num_reproducciones, artistas_similares) 
             VALUES (%s, %s, %s, %s, %s)"""
    valores = (row['artista'], row['biografia'], row['Oyentes'], row['N_Reproducciones'], row['artistas_similares'])
    
    try:
        cursor.execute(sql, valores)
    except Exception as e:
        print(f"Error con artista {row['artista']}: {e}")

conexion.commit()
print("Artistas insertados.")

Error con artista BTS: 1264 (22003): Out of range value for column 'num_reproducciones' at row 1
Error con artista Taylor Swift: 1264 (22003): Out of range value for column 'num_reproducciones' at row 1
Artistas insertados.


In [ ]:
# 1. Crear mapas de Nombres -> IDs
cursor.execute("SELECT id_genero, nombre_genero FROM generos")
mapa_generos = {nombre: id_gen for (id_gen, nombre) in cursor.fetchall()}

cursor.execute("SELECT id_artista, nombre FROM artistas")
mapa_artistas = {nombre: id_art for (id_art, nombre) in cursor.fetchall()}

# 2. Insertar canciones mapeando los nombres a sus IDs
for _, row in df_canciones.iterrows():
    # Buscamos los IDs correspondientes
    id_gen = mapa_generos.get(row['genre'])
    id_art = mapa_artistas.get(row['name_artist'])
    
    # Solo insertamos si encontramos al artista (para mantener integridad)
    if id_art and id_gen:
        sql = "INSERT INTO canciones (nombre, anio, id_genero, id_artista) VALUES (%s, %s, %s, %s)"
        cursor.execute(sql, (row['name_track'], row['year'], id_gen, id_art))

conexion.commit()
cursor.close()
conexion.close()
print("Proceso finalizado con éxito.")